## This process is similar with non-Reverse notebook process, while it does calculate before time

In [104]:
import pandas as pd
# import pandasql as ps #maybe needed
import ssl
import numpy as np
from datetime import datetime,timedelta #to read and change time
import sys #allow print full np.array

PS: depends on how many threads you can start in you cpu

#### if you have already the files, start from here, above is for showing prof web scraping skills

In [2]:
# for if you have already the files, start from here
_2015min = pd.read_csv('2015min.csv')
_2016min = pd.read_csv('2016min.csv')
_2017min = pd.read_csv('2017min.csv')
_2018min = pd.read_csv('2018min.csv')
_2019min = pd.read_csv('2019min.csv')
_2020min = pd.read_csv('2020min.csv')
_2021min = pd.read_csv('2021min.csv')

In [5]:
# show prof. in this project, our newest data time is '2021-07-22 00:04:00' if dont change the attached csv given
_2021min.head()

,Unnamed: 0,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,0,1626912240000,2021-07-22 00:04:00,BTCUSD,31979.61,32007.58,31974.32,31980.59,0.107347
1,1,1626912180000,2021-07-22 00:03:00,BTCUSD,31962.79,31979.61,31868.75,31979.61,2.634666
2,2,1626912120000,2021-07-22 00:02:00,BTCUSD,32020.05,32028.77,31946.06,31962.79,1.128569
3,3,1626912060000,2021-07-22 00:01:00,BTCUSD,32096.50,32096.50,32020.05,32020.05,1.401139
4,4,1626912000000,2021-07-22 00:00:00,BTCUSD,32142.86,32156.91,32096.50,32096.50,0.026117


In [5]:
# concat all csv 
# _1min means 1min interval
_1min = pd.concat([_2021min,_2020min,_2019min,_2018min,_2017min,_2016min,_2015min])
_1min.to_csv('1min.csv')
_1min.reset_index(drop=True, inplace=True) #because concat from different files, so need to reset index
_1min.head()

,Unnamed: 0,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,0,1626912240000,2021-07-22 00:04:00,BTCUSD,31979.61,32007.58,31974.32,31980.59,0.107347
1,1,1626912180000,2021-07-22 00:03:00,BTCUSD,31962.79,31979.61,31868.75,31979.61,2.634666
2,2,1626912120000,2021-07-22 00:02:00,BTCUSD,32020.05,32028.77,31946.06,31962.79,1.128569
3,3,1626912060000,2021-07-22 00:01:00,BTCUSD,32096.50,32096.50,32020.05,32020.05,1.401139
4,4,1626912000000,2021-07-22 00:00:00,BTCUSD,32142.86,32156.91,32096.50,32096.50,0.026117


In [2]:
# _1min = pd.read_csv('1min.csv')

In [3]:
#we only use date close volume, therefore extract these 3 columns only
btc_time_close_volume = _1min[['Date','Close','Volume']]
btc_time_close_volume.reset_index(drop=True, inplace=True)
btc_time_close_volume

,Date,Close,Volume
0,2021-07-22 00:04:00,31980.59,0.107347
1,2021-07-22 00:03:00,31979.61,2.634666
2,2021-07-22 00:02:00,31962.79,1.128569
3,2021-07-22 00:01:00,32020.05,1.401139
4,2021-07-22 00:00:00,32096.50,0.026117
...,...,...,...
2938661,2015-10-08 13:44:00,242.96,0.033491
2938662,2015-10-08 13:43:00,242.96,0.010000
2938663,2015-10-08 13:42:00,242.95,0.000000
2938664,2015-10-08 13:41:00,242.95,0.001000


In [9]:
btc_time_close_volume.to_csv('btc_time_close_volume.csv')

In [4]:
# time in min, used for slicing
_1d=24*60
_12h=12*60
_6h=6*60
_3h=3*60
_1h=60
_30m=30
_10m=10
_1m=1 #not for analyzing, but for the nan row, will see that below

In [137]:
asc_p=list(btc_time_close_volume['Close']) 
asc_p_arr = np.array(asc_p) #change to array first so is faster

In [149]:
def cal_returns_reverse(time, name): #_p is price #function used for calculating returns
    shift_p = asc_p_arr[time:] #shifted price list
    now_p = asc_p_arr[:len(asc_p)-time] #now price list
    return_list = np.multiply(np.divide(np.subtract(now_p, shift_p), now_p),100) #new-old/old*100% formula for returns
    return pd.DataFrame({'return% '+name : return_list}) #return df

In [150]:
#put together

# ~5sec
returns_df = pd.concat(
    [
    cal_returns_reverse(_1m,'1m_r'),
    cal_returns_reverse(_10m,'10m_r'),
    cal_returns_reverse(_30m,'30m_r'),
    cal_returns_reverse(_1h,'1h_r'),
    cal_returns_reverse(_3h,'3h_r'),
    cal_returns_reverse(_6h,'6h_r'),
    cal_returns_reverse(_12h,'12h_r'),
    cal_returns_reverse(_1d,'1d_r'),
    ],
    axis=1
)


In [151]:
# important
print(returns_df.shape,btc_time_close_volume.shape) 

returns_df.head(11)

(2938665, 8) (2938666, 3)


,return% 1m_r,return% 10m_r,return% 30m_r,return% 1h_r,return% 3h_r,return% 6h_r,return% 12h_r,return% 1d_r
0,0.003064,-0.584136,-0.394771,-0.114382,0.666404,-0.612090,1.465076,7.022822
1,0.052596,-0.560451,-0.354320,-0.295939,0.646506,-0.592440,1.416715,7.075602
2,-0.179146,-0.594035,-0.415358,-0.344651,0.601324,-0.516976,1.395560,6.758546
3,-0.238757,-0.419175,-0.240974,-0.103997,0.709118,-0.200968,1.624139,6.786810
4,-0.144439,-0.166716,0.042840,0.100634,0.807284,-0.092938,1.772280,7.262910
5,0.108484,-0.072956,0.259747,0.089196,1.051089,-0.148804,1.811382,7.297515
6,-0.013548,-0.306684,0.129096,-0.021397,0.780678,-0.219509,1.605083,7.181515
7,-0.027653,-0.260710,0.161340,0.016318,0.720689,-0.186751,1.647093,7.192905
8,-0.015784,-0.279161,0.231685,0.110799,0.757132,-0.147161,1.675715,7.226780
9,-0.127964,-0.222870,0.260316,0.275724,0.629702,-0.090705,1.815927,7.250168


In [147]:
returns_df

,return% 1m_r,return% 10m_r,return% 30m_r,return% 1h_r,return% 3h_r,return% 6h_r,return% 12h_r,return% 1d_r
0,0.003064,-0.584136,-0.394771,-0.114382,0.666404,-0.612090,1.465076,7.022822
1,0.052596,-0.560451,-0.354320,-0.295939,0.646506,-0.592440,1.416715,7.075602
2,-0.179146,-0.594035,-0.415358,-0.344651,0.601324,-0.516976,1.395560,6.758546
3,-0.238757,-0.419175,-0.240974,-0.103997,0.709118,-0.200968,1.624139,6.786810
4,-0.144439,-0.166716,0.042840,0.100634,0.807284,-0.092938,1.772280,7.262910
...,...,...,...,...,...,...,...,...
2938660,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2938661,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2938662,0.004116,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2938663,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
returns_df_back = returns_df
returns_df_back.reset_index(drop=True, inplace=True)
returns_df_back

,return% 1m_r,return% 10m_r,return% 30m_r,return% 1h_r,return% 3h_r,return% 6h_r,return% 12h_r,return% 1d_r
0,0.003064,-0.584136,-0.394771,-0.114382,0.666404,-0.612090,1.465076,7.022822
1,0.052596,-0.560451,-0.354320,-0.295939,0.646506,-0.592440,1.416715,7.075602
2,-0.179146,-0.594035,-0.415358,-0.344651,0.601324,-0.516976,1.395560,6.758546
3,-0.238757,-0.419175,-0.240974,-0.103997,0.709118,-0.200968,1.624139,6.786810
4,-0.144439,-0.166716,0.042840,0.100634,0.807284,-0.092938,1.772280,7.262910
...,...,...,...,...,...,...,...,...
2938660,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2938661,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2938662,0.004116,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2938663,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
# ~10s
#concat and save
btc_time_close_volume_ret = pd.concat([btc_time_close_volume, returns_df_back], axis=1)
# btc_time_close_volume_ret.to_csv('btc_time_close_volume_ret.csv')
btc_time_close_volume_ret

,Date,Close,Volume,return% 1m_r,return% 10m_r,return% 30m_r,return% 1h_r,return% 3h_r,return% 6h_r,return% 12h_r,return% 1d_r
0,2021-07-22 00:04:00,31980.59,0.107347,0.003064,-0.584136,-0.394771,-0.114382,0.666404,-0.612090,1.465076,7.022822
1,2021-07-22 00:03:00,31979.61,2.634666,0.052596,-0.560451,-0.354320,-0.295939,0.646506,-0.592440,1.416715,7.075602
2,2021-07-22 00:02:00,31962.79,1.128569,-0.179146,-0.594035,-0.415358,-0.344651,0.601324,-0.516976,1.395560,6.758546
3,2021-07-22 00:01:00,32020.05,1.401139,-0.238757,-0.419175,-0.240974,-0.103997,0.709118,-0.200968,1.624139,6.786810
4,2021-07-22 00:00:00,32096.50,0.026117,-0.144439,-0.166716,0.042840,0.100634,0.807284,-0.092938,1.772280,7.262910
...,...,...,...,...,...,...,...,...,...,...,...
2938661,2015-10-08 13:44:00,242.96,0.033491,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2938662,2015-10-08 13:43:00,242.96,0.010000,0.004116,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2938663,2015-10-08 13:42:00,242.95,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2938664,2015-10-08 13:41:00,242.95,0.001000,0.185223,NaN,NaN,NaN,NaN,NaN,NaN,NaN


return part is finished

##### now start vol of diff time intervals

the steps are similar to return

In [73]:
asc_v_np=np.array(btc_time_close_volume['Volume'])
#p is price, v is vol

In [18]:
# print(np.arange(10),end='\n\n')
# testingtestingtesting=np.lib.stride_tricks.sliding_window_view(np.arange(10),5)
# print(testingtestingtesting)
# np.sum(testingtestingtesting ,axis=1)
# same for np.std, i dont show again here

In [74]:
def cal_volume_reverse(time, name): #_v is price #function used for calculating volume within time intervals
    get_all_shifted_2Darray= np.lib.stride_tricks.sliding_window_view(asc_v_np, time) #hard to explain, just see the above cell and output
    sumed = np.sum(get_all_shifted_2Darray ,axis=1)
    return pd.DataFrame({'volume '+name : sumed}) #return df

In [75]:
#put together

# ~5sec
volume_df = pd.concat(
    [
    cal_volume_reverse(_1m,'1m_r'),
    cal_volume_reverse(_10m,'10m_r'),
    cal_volume_reverse(_30m,'30m_r'),
    cal_volume_reverse(_1h,'1h_r'),
    cal_volume_reverse(_3h,'3h_r'),
    cal_volume_reverse(_6h,'6h_r'),
    cal_volume_reverse(_12h,'12h_r'),
    cal_volume_reverse(_1d,'1d_r'),
    ],
    axis=1
)

no need to add one row, because volume in 1min is not return in 1min, no need to get future data, therefore no NaN

then just do the same thing same as above, except adding rows

In [67]:
# important
print(volume_df.shape,btc_time_close_volume.shape) 

(2938666, 8) (2938666, 3)


In [79]:
volume_df_back = volume_df
volume_df_back.head(20)

,volume 1m_r,volume 10m_r,volume 30m_r,volume 1h_r,volume 3h_r,volume 6h_r,volume 12h_r,volume 1d_r
0,0.107347,5.639999,26.067276,57.835534,144.292388,1296.719646,2216.899517,3120.495970
1,2.634666,6.836184,26.185899,59.502019,144.190214,1297.047511,2217.136943,3120.500567
2,1.128569,4.464224,23.620209,57.347867,142.266663,1295.641593,2215.101002,3118.568191
3,1.401139,3.365116,22.575227,56.282312,141.851223,1298.636700,2214.292021,3117.723341
4,0.026117,2.146406,21.418573,54.917326,140.582084,1298.588791,2213.366869,3116.458713
5,0.009594,3.061754,24.191985,57.100948,141.302708,1303.490822,2215.666472,3116.931556
6,0.189189,3.263488,24.489616,58.174580,146.563654,1305.529299,2216.035155,3117.176604
7,0.061549,5.804689,24.333237,57.999120,146.594742,1306.681410,2216.211052,3117.268666
8,0.059241,7.159741,24.709449,58.225846,147.526202,1310.161771,2216.831226,3117.760560
9,0.022589,7.397511,24.744749,58.233716,151.798018,1310.687909,2221.448585,3117.714192


return + volume part is finished
##### now start volatility(sd of return) of diff time intervals
the steps are similar

i will only explain the steps which are different than volume and return

In [113]:
# actually everything is the same, just different in getting
asc_r_1m_arr=np.array(returns_df_back['return% 1m_r']) 

def cal_volat_reverse(time, name): #_v is price #function used for calculating volume within time intervals
    get_all_shifted_2Darray= np.lib.stride_tricks.sliding_window_view(asc_r_1m_arr, time) #hard to explain, just see the above code and output
    stded = np.std(get_all_shifted_2Darray ,axis=1)
    return pd.DataFrame({'volat '+name : stded}) #return df

In [114]:
sd1m = cal_volat_reverse(_1m,'1m_r')

In [115]:
sd10m = cal_volat_reverse(_10m,'10m_r')

In [116]:
sd30m = cal_volat_reverse(_30m,'30m_r')

need to be <mark style="background-color: yellow">VERY BECAREFUL WHEN RUNNING BELOW CELLS</mark>. i chose to run them separately because i can see which part use how much time also i dont why, but it seems to run much faster if i separate

the below number are time my computer needs (i7-7700 + 32RAM + x64), may cause more time

sd1h=4s
sd3h=6s
sd6h=10+s
sd12h=20s
sd1d=15m+

In [117]:
sd1h = cal_volat_reverse(_1h,'1h_r')

In [118]:
sd3h = cal_volat_reverse(_3h,'3h_r')

In [120]:
%%time
sd6h = cal_volat_reverse(_6h,'6h_r')

Wall time: 7 s


In [32]:
%%time
sd12h=cal_volat_reverse(_12h,'12h_r')

think before running this if your spec is not good.
took my pc 15mins and not yet finish

In [ ]:
# sd1d=    cal_volat(_1d,'1d')

In [121]:
#put together

# ~5sec
volat_df = pd.concat(
    [
    sd1m,
    sd10m,
    sd30m,
    sd1h,
    sd3h,
    sd6h,
    sd12h
    ],
    axis=1
)

In [122]:
volat_df_back = volat_df
volat_df_back.reset_index(drop=True, inplace=True)
volat_df_back.head(20)

,volat 1m_r,volat 10m_r,volat 30m_r,volat 1h_r,volat 3h_r,volat 6h_r
0,0.0,0.103957,0.082646,0.083530,0.083327,0.172350
1,0.0,0.105575,0.083224,0.086521,0.083341,0.172354
2,0.0,0.102575,0.082366,0.086206,0.083261,0.172465
3,0.0,0.095272,0.076435,0.083600,0.082324,0.172361
4,0.0,0.069784,0.063818,0.077836,0.081021,0.172040
5,0.0,0.057145,0.059004,0.078207,0.080572,0.172195
6,0.0,0.052548,0.056234,0.076956,0.081192,0.172111
7,0.0,0.055741,0.056191,0.077000,0.081390,0.172113
8,0.0,0.056066,0.056235,0.077380,0.081357,0.172108
9,0.0,0.059692,0.056071,0.079609,0.082089,0.172120


In [154]:
btc_everything = pd.concat(
    [btc_time_close_volume_ret,
     volume_df_back,
     volat_df_back
    ],
    axis=1
)

In [155]:
btc_everything.head()

,Date,Close,Volume,return% 1m_r,return% 10m_r,return% 30m_r,return% 1h_r,return% 3h_r,return% 6h_r,return% 12h_r,...,volume 3h_r,volume 6h_r,volume 12h_r,volume 1d_r,volat 1m_r,volat 10m_r,volat 30m_r,volat 1h_r,volat 3h_r,volat 6h_r
0,2021-07-22 00:04:00,31980.59,0.107347,0.003064,-0.584136,-0.394771,-0.114382,0.666404,-0.612090,1.465076,...,144.292388,1296.719646,2216.899517,3120.495970,0.0,0.103957,0.082646,0.083530,0.083327,0.172350
1,2021-07-22 00:03:00,31979.61,2.634666,0.052596,-0.560451,-0.354320,-0.295939,0.646506,-0.592440,1.416715,...,144.190214,1297.047511,2217.136943,3120.500567,0.0,0.105575,0.083224,0.086521,0.083341,0.172354
2,2021-07-22 00:02:00,31962.79,1.128569,-0.179146,-0.594035,-0.415358,-0.344651,0.601324,-0.516976,1.395560,...,142.266663,1295.641593,2215.101002,3118.568191,0.0,0.102575,0.082366,0.086206,0.083261,0.172465
3,2021-07-22 00:01:00,32020.05,1.401139,-0.238757,-0.419175,-0.240974,-0.103997,0.709118,-0.200968,1.624139,...,141.851223,1298.636700,2214.292021,3117.723341,0.0,0.095272,0.076435,0.083600,0.082324,0.172361
4,2021-07-22 00:00:00,32096.50,0.026117,-0.144439,-0.166716,0.042840,0.100634,0.807284,-0.092938,1.772280,...,140.582084,1298.588791,2213.366869,3116.458713,0.0,0.069784,0.063818,0.077836,0.081021,0.172040


In [156]:
# save #1+min
btc_everything.to_csv('btc_everything_reverse.csv')

In [157]:
a=pd.read_csv('btc_everything_reverse.csv')
a

,Unnamed: 0,Date,Close,Volume,return% 1m_r,return% 10m_r,return% 30m_r,return% 1h_r,return% 3h_r,return% 6h_r,...,volume 3h_r,volume 6h_r,volume 12h_r,volume 1d_r,volat 1m_r,volat 10m_r,volat 30m_r,volat 1h_r,volat 3h_r,volat 6h_r
0,0,2021-07-22 00:04:00,31980.59,0.107347,0.003064,-0.584136,-0.394771,-0.114382,0.666404,-0.612090,...,144.292388,1296.719646,2216.899517,3120.495970,0.0,0.103957,0.082646,0.083530,0.083327,0.172350
1,1,2021-07-22 00:03:00,31979.61,2.634666,0.052596,-0.560451,-0.354320,-0.295939,0.646506,-0.592440,...,144.190214,1297.047511,2217.136943,3120.500567,0.0,0.105575,0.083224,0.086521,0.083341,0.172354
2,2,2021-07-22 00:02:00,31962.79,1.128569,-0.179146,-0.594035,-0.415358,-0.344651,0.601324,-0.516976,...,142.266663,1295.641593,2215.101002,3118.568191,0.0,0.102575,0.082366,0.086206,0.083261,0.172465
3,3,2021-07-22 00:01:00,32020.05,1.401139,-0.238757,-0.419175,-0.240974,-0.103997,0.709118,-0.200968,...,141.851223,1298.636700,2214.292021,3117.723341,0.0,0.095272,0.076435,0.083600,0.082324,0.172361
4,4,2021-07-22 00:00:00,32096.50,0.026117,-0.144439,-0.166716,0.042840,0.100634,0.807284,-0.092938,...,140.582084,1298.588791,2213.366869,3116.458713,0.0,0.069784,0.063818,0.077836,0.081021,0.172040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2938661,2938661,2015-10-08 13:44:00,242.96,0.033491,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2938662,2938662,2015-10-08 13:43:00,242.96,0.010000,0.004116,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2938663,2938663,2015-10-08 13:42:00,242.95,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2938664,2938664,2015-10-08 13:41:00,242.95,0.001000,0.185223,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
